# Explore JWST and HST images

In this notebook, we load Imviz and we use mainly the user interface to explore some JWST and HST images. [The data can be downloaded from Box](https://stsci.box.com/s/fui7ufapngb7qkn60ep0u79e5geb0cz0) (images are quite large so they are not packaged into a single humongous zip file. Feel free to download as many as you like.

## System requirements
First, we create an environment with jdaviz which contains all the spectroscopic packages we need.

`conda create -n jdaviz python`<br>
`conda activate jdaviz`<br>
`pip install jdaviz`<br>
If you wish to use the footprints from JWST, you need to install the pysiaf package too<br>
`pip install pysiaf`

In [ ]:
import warnings
from jdaviz import Imviz
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture
from regions import RectangleSkyRegion
import astropy.units as u

Choose one of the two ways to run Imviz.

In [ ]:
# If in jupyter lab
imviz = Imviz()
imviz.show(loc='sidecar', height=1000)

## Use the GUI programmatically

Change path to your local directory if you are not working on the platform.

In [ ]:
data_dir = '/home/shared/preloaded-fits/jdaviz_data/UNCOVER/'
#data_dir = './UNCOVER/'

files = ['jw02561-o001_t003_nircam_clear-f200w_i2d.fits',
         'jw02561-o001_t003_nircam_clear-f444w_i2d.fits',
         'ic8n06020_drz.fits'
        ]

labels = ['JWST F200W',
          'JWST F444W',
          'HST'
         ]

for i in range(0,len(files)):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(f'{data_dir}/{files[i]}', data_label=labels[i])

In [ ]:
viewer = imviz.default_viewer

In [ ]:
imviz.plugins

### Link by WCS

In [ ]:
linkwcs = imviz.plugins['Links Control']
linkwcs.link_type = 'WCS'

### Set better plot options

In [ ]:
plotopt = imviz.plugins['Plot Options']

plotopt.layer = 'HST[SCI,1]'
plotopt.stretch_function = 'Logarithmic'
plotopt.stretch_vmin = 0.52
plotopt.stretch_vmax = 30
plotopt.image_colormap = 'Gray'

plotopt.layer = 'JWST F200W[DATA]'
plotopt.stretch_function = 'Logarithmic'
plotopt.stretch_vmin = 0.12
plotopt.stretch_vmax = 20
plotopt.image_colormap = 'Viridis'

plotopt.layer = 'JWST F444W[DATA]'
plotopt.stretch_function = 'Logarithmic'
plotopt.stretch_vmin = 0.24
plotopt.stretch_vmax = 20
plotopt.image_colormap = 'Inferno'


Now to blink between the images you can press the 'b' key when the mouse is on the viewer. Land on the F444W filter for the next cell.

### Spot an object that is not in the HST data

In [ ]:
# Center the image on given sky coordinates.
#sky = SkyCoord(ra=3.57984*u.degree, dec=-30.40156*u.degree, frame='icrs')
sky = SkyCoord(ra=3.5757827*u.degree, dec=-30.4130878*u.degree, frame='icrs')
viewer.center_on(sky)

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 3

### Select a subset around the object

In [ ]:
interesting_object = SkyCircularAperture(sky, 0.5 * u.arcsec)
imviz.load_regions(interesting_object)

### Get aperture photometry for the object

In [ ]:
aperphot = imviz.plugins['Imviz Simple Aperture Photometry']
aperphot.open_in_tray()

### Load IFU region
The NIRSpec IFU is a 3''x3'' square.

In [ ]:
region = RectangleSkyRegion(sky, 3*u.arcsec, 3*u.arcsec, angle=30*u.deg)

footprint = imviz.plugins['Footprints']
footprint.color = '#02d6fC'
footprint.import_region(region)

### Measure a distance
Open the Markers plugin and mark 2 coordinates using the mouse and the 'm' key.

In [ ]:
markers = imviz.plugins['Markers']
table = markers.export_table()
table

In [ ]:
mark1 = SkyCoord(ra=table[0]['world'][0]*u.deg, dec=table[0]['world'][1]*u.deg)
mark2 = SkyCoord(ra=table[1]['world'][0]*u.deg, dec=table[1]['world'][1]*u.deg)
print("Distance between two markers:")
print(mark1.separation(mark2).arcsecond, 'arcsec')